In [1]:
# EDA
import pandas as pd
import numpy as np

# oracle sql
import cx_Oracle

## date handling
from datetime import datetime, timedelta
from pytz import timezone


# visualizing
import seaborn as sns
import matplotlib.pyplot as plt

# modelling
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.ensemble import RandomForestRegressor

# accuracy metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# scaling
from sklearn.preprocessing import StandardScaler

# deployment
import pickle as pkl
from sklearn.pipeline import Pipeline

## for ignoring deprecation warnings
from warnings import filterwarnings
filterwarnings('ignore')

## Create Connection

In [23]:
dsn = cx_Oracle.makedsn("dwh-scan.kapitalbank.az", 1521, service_name="report")

connection = cx_Oracle.connect(user='SAMADOVIAZ_15121[FRAUD]',password='Dwh*@(#052316:09B1CD62470',dsn=dsn)

In [24]:
query_all="""
select /*+parallel */
 a.bank_time,
 a.TERMOWNER,
 round(a.CASHBACK_RATE, 1) CASHBACK_RATE,
 round(a.CASHBACK_AMOUNT, 1) CASHBACK_AMOUNT,
 round(a.BALANCE_AFTER, 1) BALANCE_AFTER,
 a.DIRECTION,
 b.marstat,
 b.birth_date,
 b.hiredate,
 c.mcc,
 c.mcc_description,
 c.mcc_group,
 d.id,
 d.currency,
 d.name
  from cms.f_cashback_transaction a
 inner join fraud.emp_01_05_2023 b
    on a.cms_id = b.cms_id
 INNER JOIN CMS.D_MCC_DETAILS C
    ON A.MCC = C.MCC
 inner join dwh.D_CURRENCY d
    on a.from_cur = d.id"""

In [25]:
%%time

df_all=pd.read_sql(query_all,con=connection)
df_all.tail()

Wall time: 1min 40s


,BANK_TIME,TERMOWNER,CASHBACK_RATE,CASHBACK_AMOUNT,BALANCE_AFTER,DIRECTION,MARSTAT,BIRTH_DATE,HIREDATE,MCC,MCC_DESCRIPTION,MCC_GROUP,ID,CURRENCY,NAME
2067836,2021-09-16 17:43:03,SPECIALTY COFFEE,1.5,0.1,0.3,CREDIT,Evli,25.06.1967,17.10.2019,5499,"Misc Food Stores-Speciality,Convenience,",RETAIL STORES,944,AZN,Азербайджанский манат
2067837,2021-09-16 13:43:05,POS LINS COFFEE S.R,1.5,0.1,0.1,CREDIT,Evli,25.06.1967,17.10.2019,5812,"Eating Places, Restaurants",VARIOUS SHOPS,944,AZN,Азербайджанский манат
2067838,2021-09-16 11:49:07,PASHA POS H2H MCD 28 MAY 6,1.5,0.0,0.0,CREDIT,Evli,25.06.1967,17.10.2019,5814,Fast Food Resturant,VARIOUS SHOPS,944,AZN,Азербайджанский манат
2067839,2021-09-16 11:51:28,BRAVO MARKET 28 MALL,1.5,0.0,0.1,CREDIT,Evli,25.06.1967,17.10.2019,5411,"Grocery Stores, Supermarkets",RETAIL STORES,944,AZN,Азербайджанский манат
2067840,2023-04-26 22:54:29,L?MON SUPERMARKET,1.5,0.2,2.5,CREDIT,Evli,25.06.1967,17.10.2019,5411,"Grocery Stores, Supermarkets",RETAIL STORES,944,AZN,Азербайджанский манат


In [27]:
df_all.head()

,BANK_TIME,TERMOWNER,CASHBACK_RATE,CASHBACK_AMOUNT,BALANCE_AFTER,DIRECTION,MARSTAT,BIRTH_DATE,HIREDATE,MCC,MCC_DESCRIPTION,MCC_GROUP,ID,CURRENCY,NAME
0,2022-12-19 18:31:21,AL MARKET XATAI NASRA,3.0,0.3,50.3,CREDIT,Subay,21.01.2000,26.12.2022,5411,"Grocery Stores, Supermarkets",RETAIL STORES,944,AZN,Азербайджанский манат
1,2022-12-18 13:41:43,ZEYTUN APTEK 12,1.5,0.3,49.9,CREDIT,Subay,21.01.2000,26.12.2022,5912,"Drug Stores, Pharmacies",VARIOUS SHOPS,944,AZN,Азербайджанский манат
2,2022-12-18 13:38:05,BUTA FARM APTEK 324,1.5,0.7,49.6,CREDIT,Subay,21.01.2000,26.12.2022,5912,"Drug Stores, Pharmacies",VARIOUS SHOPS,944,AZN,Азербайджанский манат
3,2022-11-09 15:40:43,AL MARKET XATAI NASRA,3.0,0.0,35.5,CREDIT,Subay,21.01.2000,26.12.2022,5411,"Grocery Stores, Supermarkets",RETAIL STORES,944,AZN,Азербайджанский манат
4,2023-04-19 21:15:02,BUTA FARM APTEK 324,1.5,0.1,29.7,CREDIT,Subay,21.01.2000,26.12.2022,5912,"Drug Stores, Pharmacies",VARIOUS SHOPS,944,AZN,Азербайджанский манат


In [30]:
# df_all.to_csv(r"C:\Users\SamadovIAz\Desktop\Cashback_Analyse\df.csv", sep=',', encoding='utf-8', header='true')

In [ ]:
df_all.to_parquet(r"C:\Users\SamadovIAz\Desktop\Cashback_Analyse\df.parquet")  # 396,204 KB csv   --> 35,963 KB parquet

In [31]:
check=pd.read_parquet('df.parquet')

In [34]:
check

,BANK_TIME,TERMOWNER,CASHBACK_RATE,CASHBACK_AMOUNT,BALANCE_AFTER,DIRECTION,MARSTAT,BIRTH_DATE,HIREDATE,MCC,MCC_DESCRIPTION,MCC_GROUP,ID,CURRENCY,NAME
0,2022-12-19 18:31:21,AL MARKET XATAI NASRA,3.0,0.3,50.3,CREDIT,Subay,21.01.2000,26.12.2022,5411,"Grocery Stores, Supermarkets",RETAIL STORES,944,AZN,Азербайджанский манат
1,2022-12-18 13:41:43,ZEYTUN APTEK 12,1.5,0.3,49.9,CREDIT,Subay,21.01.2000,26.12.2022,5912,"Drug Stores, Pharmacies",VARIOUS SHOPS,944,AZN,Азербайджанский манат
2,2022-12-18 13:38:05,BUTA FARM APTEK 324,1.5,0.7,49.6,CREDIT,Subay,21.01.2000,26.12.2022,5912,"Drug Stores, Pharmacies",VARIOUS SHOPS,944,AZN,Азербайджанский манат
3,2022-11-09 15:40:43,AL MARKET XATAI NASRA,3.0,0.0,35.5,CREDIT,Subay,21.01.2000,26.12.2022,5411,"Grocery Stores, Supermarkets",RETAIL STORES,944,AZN,Азербайджанский манат
4,2023-04-19 21:15:02,BUTA FARM APTEK 324,1.5,0.1,29.7,CREDIT,Subay,21.01.2000,26.12.2022,5912,"Drug Stores, Pharmacies",VARIOUS SHOPS,944,AZN,Азербайджанский манат
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2067836,2021-09-16 17:43:03,SPECIALTY COFFEE,1.5,0.1,0.3,CREDIT,Evli,25.06.1967,17.10.2019,5499,"Misc Food Stores-Speciality,Convenience,",RETAIL STORES,944,AZN,Азербайджанский манат
2067837,2021-09-16 13:43:05,POS LINS COFFEE S.R,1.5,0.1,0.1,CREDIT,Evli,25.06.1967,17.10.2019,5812,"Eating Places, Restaurants",VARIOUS SHOPS,944,AZN,Азербайджанский манат
2067838,2021-09-16 11:49:07,PASHA POS H2H MCD 28 MAY 6,1.5,0.0,0.0,CREDIT,Evli,25.06.1967,17.10.2019,5814,Fast Food Resturant,VARIOUS SHOPS,944,AZN,Азербайджанский манат
2067839,2021-09-16 11:51:28,BRAVO MARKET 28 MALL,1.5,0.0,0.1,CREDIT,Evli,25.06.1967,17.10.2019,5411,"Grocery Stores, Supermarkets",RETAIL STORES,944,AZN,Азербайджанский манат
